Импортируем необходимые библиотеки:

In [ ]:
import numpy as np
import os
import shutil
from keras import layers, models, optimizers
from keras.applications import MobileNet
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input
from keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

Загрузка датасета и его распаковка:

In [ ]:
!wget https://storage.yandexcloud.net/academy.ai/cat-and-dog.zip
!unzip -qo "cat-and-dog" -d ./temp

--2025-04-13 18:17:10--  https://storage.yandexcloud.net/academy.ai/cat-and-dog.zip
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228082266 (218M) [application/x-zip-compressed]
Saving to: ‘cat-and-dog.zip’

cat-and-dog.zip     100%[===================>] 217.52M  32.2MB/s    in 7.9s    

2025-04-13 18:17:19 (27.7 MB/s) - ‘cat-and-dog.zip’ saved [228082266/228082266]



Создадим некоторые переменные для дальнейшей работы с изображениями в датасете:

In [ ]:
IMG_WIDTH, IMG_HEIGHT = 150, 150 #параметры изображений

IMAGE_PATH = './temp/training_set/training_set/' #определение путей к данным
BASE_DIR = './dataset/'

CLASS_LIST = sorted(os.listdir(IMAGE_PATH)) #анализ классов данных
CLASS_COUNT = len(CLASS_LIST)
NUM_CLASSES = CLASS_COUNT

Выполним стандартную практику в машинном обучении - определим файловую структуру:

In [ ]:
if os.path.exists(BASE_DIR): #удаляем старую папку, если существует
    shutil.rmtree(BASE_DIR)

os.mkdir(BASE_DIR) #создание новой базовой директории

#создаем подпапок
train_dir = os.path.join(BASE_DIR, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(BASE_DIR, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(BASE_DIR, 'test')
os.mkdir(test_dir)

Создадим функцию создания подвыборок (папок с файлами). В этой функции используется `shutil.copyfile()` — метод модуля *shutil* в Python, который используется для копирования содержимого исходного файла в целевой. При этом метаданные файла не копируются.

In [ ]:
def create_dataset(img_path, new_path, class_name, start_index, end_index):
    src_path = os.path.join(img_path, class_name)
    dst_path = os.path.join(new_path, class_name)
    os.mkdir(dst_path)

    class_files = os.listdir(src_path)
    for fname in class_files[start_index:end_index]:
        src = os.path.join(src_path, fname)
        dst = os.path.join(dst_path, fname)
        shutil.copyfile(src, dst)

Далее выполним разделение исходного датасета на тренировочную, валидационную и тестовую выборки для каждого класса изображений:

In [ ]:
#распределение изображений
for class_label in range(CLASS_COUNT):
    class_name = CLASS_LIST[class_label]
    class_files = os.listdir(os.path.join(IMAGE_PATH, class_name))
    total_samples = len(class_files)

    train_end = int(total_samples * 0.6)
    validation_end = train_end + int(total_samples * 0.2)

    create_dataset(IMAGE_PATH, train_dir, class_name, 0, train_end) #тренировочная выборка
    create_dataset(IMAGE_PATH, validation_dir, class_name, train_end, validation_end) #валидационная выборка
    create_dataset(IMAGE_PATH, test_dir, class_name, validation_end, total_samples) #контрольная выборка

Создадим модель нейронной сети для классификации изображений с использованием предобученной **MobileNet** в качестве базового слоя. **MobileNet** — предобученная **CNN** (уже обучена на **ImageNet**).

`include_top=False` — не включаем верхние (классификационные) слои **MobileNet**

**GlobalAveragePooling2D** — заменяет классический **Flatten**, усредняя значения по каждому каналу, уменьшает количество параметров и предотвращает переобучение.

In [ ]:
#функция создания модели
def model_maker():
    base_model = MobileNet(include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))

    for layer in base_model.layers[:]:
        layer.trainable = False

    input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Dense(64, activation='relu')(custom_model)
    custom_model = Dropout(0.5)(custom_model)
    predictions = Dense(NUM_CLASSES, activation='softmax')(custom_model)

    return Model(inputs=input, outputs=predictions)

Создадим и настроим модель нейронной сети перед началом обучения. В качестве оптимизатора будем использовать **Adam**. Выберем кросс-энтропию, которая является стандартной функцией потерь для задач классификации:

In [ ]:
#инициализация и компиляция модели
model = model_maker()
model.compile(optimizer=optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

<ipython-input-7-b4310b25c3b9>:3: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNet(include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))


17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Создадим генераторы данных для тренировочного, валидационного и контрольного наборов, применяя *аугментацию данных* только к тренировочным изображениям:

In [ ]:
#создание генераторов данных с аугментацией для тренировочного набора
train_datagen = ImageDataGenerator(
    rescale=1./255, #нормализация пикселей [0,255] -> [0,1]
    rotation_range=20, #cлучайный поворот
    width_shift_range=0.2, #cлучайный сдвиг по ширине
    height_shift_range=0.2, #cлучайный сдвиг по высоте
    shear_range=0.2, #cлучайный сдвиг (искажение)
    zoom_range=0.2, #cлучайное масштабирование [0.8, 1.2]
    horizontal_flip=True, #cлучайное зеркальное отражение
    fill_mode='nearest' #заполнение новых пикселей при трансформациях
)

#без аугментации для валидации и теста
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

Следующий код создает генераторы данных, которые потоково загружают изображения из соответствующих директорий, применяя заданные преобразования и подготавливая данные для обучения нейронной сети:

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=20,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=20,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=20,
    class_mode='categorical'
)

Found 4803 images belonging to 2 classes.
Found 1601 images belonging to 2 classes.
Found 1601 images belonging to 2 classes.


**EarlyStopping** - ранняя остановка обучения. Это нужно, чтобы автоматически останавливать обучение, когда модель перестаёт улучшаться, предотвращая переобучение и экономя время. Мониторит метрику `val_accuracy` (точность на валидационных данных). Если точность не улучшается в течение `patience=3` эпох — обучение останавливается. `restore_best_weights=True` — после остановки модель возвращает веса, которые давали наилучшую `val_accuracy`, а не последние веса.

In [ ]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

Далее извлечём общее количество изображений в каждом наборе данных (тренировочном, валидационном и контрольном) из созданных ранее генераторов:

In [ ]:
#получаем количество образцов из генераторов
train_samples = train_generator.samples
validation_samples = validation_generator.samples
test_samples = test_generator.samples

Выполним обучение нейронной сети с использованием созданных генераторов данных:

In [ ]:
#обучение модели
history = model.fit(
    train_generator,
    steps_per_epoch=train_samples // 20, #количество шагов за эпоху
    epochs=30,
    validation_data=validation_generator,
    callbacks=[early_stopping],
    validation_steps=validation_samples // 20 #количество валидационных шагов
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
240/240 ━━━━━━━━━━━━━━━━━━━━ 137s 551ms/step - accuracy: 0.8435 - loss: 0.4417 - val_accuracy: 0.9588 - val_loss: 0.1045
Epoch 2/30
  1/240 ━━━━━━━━━━━━━━━━━━━━ 1:05 275ms/step - accuracy: 0.9000 - loss: 0.1186

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


240/240 ━━━━━━━━━━━━━━━━━━━━ 26s 107ms/step - accuracy: 0.9000 - loss: 0.1186 - val_accuracy: 0.9575 - val_loss: 0.1104
Epoch 3/30
240/240 ━━━━━━━━━━━━━━━━━━━━ 130s 543ms/step - accuracy: 0.9368 - loss: 0.1576 - val_accuracy: 0.9631 - val_loss: 0.0899
Epoch 4/30
240/240 ━━━━━━━━━━━━━━━━━━━━ 41s 170ms/step - accuracy: 0.9000 - loss: 0.1516 - val_accuracy: 0.9606 - val_loss: 0.0918
Epoch 5/30
240/240 ━━━━━━━━━━━━━━━━━━━━ 142s 590ms/step - accuracy: 0.9380 - loss: 0.1519 - val_accuracy: 0.9656 - val_loss: 0.0933
Epoch 6/30
240/240 ━━━━━━━━━━━━━━━━━━━━ 26s 105ms/step - accuracy: 0.9500 - loss: 0.1195 - val_accuracy: 0.9644 - val_loss: 0.0962
Epoch 7/30
240/240 ━━━━━━━━━━━━━━━━━━━━ 127s 529ms/step - accuracy: 0.9409 - loss: 0.1477 - val_accuracy: 0.9656 - val_loss: 0.0979
Epoch 8/30
240/240 ━━━━━━━━━━━━━━━━━━━━ 26s 107ms/step - accuracy: 1.0000 - loss: 0.0119 - val_accuracy: 0.9650 - val_loss: 0.0973


Выполним финальную оценку обученной модели на тестовом наборе данных, который модель никогда не видела во время обучения:

In [ ]:
#точность контрольной выборки
test_loss, test_acc = model.evaluate(test_generator, steps=test_samples // 20)
print(f'Точность контрольной выборки: {test_acc}')

80/80 ━━━━━━━━━━━━━━━━━━━━ 26s 324ms/step - accuracy: 0.9641 - loss: 0.0786
Точность контрольной выборки: 0.9700000286102295
